# Huấn luyện Mô hình Phân loại Học viên (Fixed K-Means)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.metrics import silhouette_score, davies_bouldin_score
import joblib
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# 1. Đọc dữ liệu
df = pd.read_csv('../data/training_data.csv')
print("Kích thước dữ liệu:", df.shape)

In [ ]:
# 2. Tiền xử lý dữ liệu
# Tính final_score nếu chưa có
if 'final_score' not in df.columns:
    df['final_score'] = df[['score1', 'score2', 'score3']].mean(axis=1).round(2)

# Xử lý school_name (giả lập nếu thiếu)
if 'school_name' not in df.columns:
    np.random.seed(42)
    school_types = ['Trường THPT Nguyễn Thượng Hiền', 'Trường THPT Gia Định', 'Trường THPT Nguyễn Thị Minh Khai']
    df['school_name'] = np.random.choice(school_types, size=len(df))

# Mã hóa trường học
school_order = [
    "Trường THCS Đoàn Thị Điểm", "THPT Nguyễn Hữu Cầu",
    "Trường THCS Nguyễn Du", "Trường THPT Mạc Đĩnh Chi",
    "Trường THCS Bạch Đằng", "Trường THPT Lê Quý Đôn",
    "Trường THCS Á Châu", "Trường THPT Trần Phú",
    "Trường THCS Việt Úc", "Trường THPT Phú Nhuận",
    "Trường THCS Trần Văn Ơn", "Trường THPT Bùi Thị Xuân",
    "Trường THCS Hai Bà Trưng", "Trường THPT Nguyễn Hữu Huân",
    "Trường THCS Nguyễn Hữu Thọ", "Trường THPT Nguyễn Thị Minh Khai",
    "Trường THCS Colette", "Trường THPT Gia Định",
    "Trường THCS Lê Quý Đôn", "Trường THPT Nguyễn Thượng Hiền"
]

unknown_schools = set(df['school_name'].unique()) - set(school_order)
if unknown_schools:
    school_order = list(unknown_schools) + school_order

encoder = OrdinalEncoder(categories=[school_order], handle_unknown='use_encoded_value', unknown_value=-1)
df['school_encoded'] = encoder.fit_transform(df[['school_name']])

# Chuẩn hóa dữ liệu
features = ['score1', 'score2', 'score3', 'final_score', 'school_encoded']
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df[features])

In [ ]:
# 3. Huấn luyện mô hình K-Means
k_final = 5
print(f"Tiến hành phân cụm với k = {k_final} trên toàn bộ dữ liệu...")

kmeans_final = KMeans(n_clusters=k_final, random_state=42, n_init=10)
df['cluster'] = kmeans_final.fit_predict(X_scaled)

print(f"Đã phân thành {k_final} cụm.")

In [ ]:
# 4. Gán nhãn xếp loại (Xuất sắc, Giỏi, Khá, Trung bình, Yếu)
cluster_avg_scores = df.groupby('cluster')['final_score'].mean().sort_values(ascending=False)
print("\nĐiểm trung bình của từng cụm:")
print(cluster_avg_scores)

cluster_to_rank = {}
sorted_clusters = cluster_avg_scores.index.tolist()

labels_priority = ['Xuất sắc', 'Giỏi', 'Khá', 'Trung bình', 'Yếu']

for i, cluster_id in enumerate(sorted_clusters):
    if i < len(labels_priority):
        label = labels_priority[i]
    else:
        label = labels_priority[-1]
    cluster_to_rank[cluster_id] = label

df['xep_loai'] = df['cluster'].map(cluster_to_rank)

print("\nÁnh xạ cụm sang xếp loại:")
for cluster, rank in cluster_to_rank.items():
    print(f"Cụm {cluster} (Avg: {cluster_avg_scores[cluster]:.2f}): {rank}")

# In thống kê kết quả
print("\nThống kê số lượng theo xếp loại:")
print(df['xep_loai'].value_counts())

In [ ]:
# 5. Đánh giá mô hình
silhouette = silhouette_score(X_scaled, df['cluster'])
davies_bouldin = davies_bouldin_score(X_scaled, df['cluster'])

print(f"\nĐánh giá mô hình:")
print(f"Silhouette Score: {silhouette:.4f} (Càng gần 1 càng tốt)")
print(f"Davies-Bouldin Index: {davies_bouldin:.4f} (Càng thấp càng tốt)")

In [ ]:
# 6. Lưu kết quả và artifacts
df.to_csv('../data/student_classification_results.csv', index=False, encoding='utf-8-sig')
print("\nĐã lưu kết quả vào student_classification_results.csv")

joblib.dump(kmeans_final, 'artifacts/kmeans_model.pkl')
joblib.dump(scaler, 'artifacts/scaler.pkl')
joblib.dump(encoder, 'artifacts/encoder.pkl')
joblib.dump(cluster_to_rank, 'artifacts/cluster_mapping.pkl')
print("Đã lưu các file artifacts: kmeans_model.pkl, scaler.pkl, encoder.pkl, cluster_mapping.pkl")